In [1]:
from socket import *

In [2]:
server = "localhost"
port = 8080

In [3]:
c = socket(AF_INET, SOCK_STREAM)
c.connect((server, port))

In [7]:
f = open("screenshoot/1.png", "wb")

In [8]:
c.sendall(b"1.png")
c.sendfile(f)

0

In [ ]:
c.close()